## Conseguir comentarios de paginas de facebook

In [1]:
import urllib2
import json
import datetime
import csv
import time
import pandas as pd
import os.path
import numpy as np

In [2]:
app_id = "app-id"
app_secret = "app-secret"

access_token = app_id + "|" + app_secret

In [3]:
munis = pd.read_csv('./data/municipios.csv')[:-1]
page_ids = munis['pagina de facebook']

IOError: File ./data/municipios.csv does not exist

In [67]:
def request_until_succeed(url):
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print e
            time.sleep(15)
            
            print "Error for URL %s: %s" % (url, datetime.datetime.now())

    return response.read()

In [68]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=comments&limit=%s&access_token=%s" % (num_statuses, access_token) # changed
    url = base + node + parameters

    data = json.loads(request_until_succeed(url))
    
    return data

from pprint import pprint

def processFacebookPageFeedStatus(status):
    comments = []
    created = []
    try:
        comments = comments + map(lambda x: x['message'].replace('\n', ' ').replace('\r', ''), status['comments']['data'])
        created = created + map(lambda x: x['created_time'], status['comments']['data'])
    except:
        return {'comments': [], 'created': []}
    return {'comments': comments, 'created': created}

In [69]:
def scrapeFacebookPageFeedStatus(page_id, access_token):
    df = pd.DataFrame({'comentario': [], 'creado': []})
    
    has_next_page = True
    num_processed = 0
    scrape_starttime = datetime.datetime.now()
    
    statuses = getFacebookPageFeedData(page_id, access_token, 100)

    print "Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime)

    while has_next_page:
        for status in statuses['data']:
            data = processFacebookPageFeedStatus(status)
            df_ = pd.DataFrame({'comentario': data['comments'], 'creado': data['created'],})
            df = pd.concat([df, df_])
            num_processed += 1
            if num_processed % 1000 == 0:
                print "%s Statuses Processed: %s" % (num_processed, datetime.datetime.now())

        if 'paging' in statuses.keys() and 'next' in statuses['paging'].keys():
            statuses = json.loads(request_until_succeed(statuses['paging']['next']))
        else:
            has_next_page = False

    df.to_csv('./data/%s.csv' % page_id, encoding='utf-8', index=False)
    print "\nDone!\n%s Statuses Processed in %s" % (num_processed, datetime.datetime.now() - scrape_starttime)
    num_processed = 0

In [ ]:
for page_id in page_ids:
    has_file = os.path.isfile('./data/%s.csv' % page_id)
    if not has_file:
        scrapeFacebookPageFeedStatus(page_id, access_token)